# XGBoost regression

## Single-node

![xgboost](https://upload.wikimedia.org/wikipedia/commons/6/69/XGBoost_logo.png)

In [ ]:
import os

MODEL_PATH = 'models'
if not os.path.exists(MODEL_PATH):
    os.makedirs(MODEL_PATH)
    
numeric_feat = [
    'pickup_weekday', 
    'pickup_weekofyear', 
    'pickup_hour', 
    'pickup_week_hour', 
    'pickup_minute', 
    'passenger_count',
]
categorical_feat = [
    'PULocationID', 
    'DOLocationID',
]
features = numeric_feat + categorical_feat
y_col = 'tip_fraction'

# Load data and feature engineering

Load a sample from a single month for this exercise

In [ ]:
import os
import numpy as np
import datetime
import pandas as pd
import s3fs
import warnings
warnings.simplefilter("ignore")

s3 = s3fs.S3FileSystem(anon=True)

taxi = pd.read_csv(
    s3.open('s3://nyc-tlc/trip data/yellow_tripdata_2019-01.csv', mode='rb'),
    parse_dates=['tpep_pickup_datetime', 'tpep_dropoff_datetime']
).sample(frac=0.3, replace=False)

In [ ]:
print(f'Num rows: {len(taxi)}, Size: {taxi.memory_usage(deep=True).sum() / 1e6} MB')

In [ ]:
def prep_df(df: pd.DataFrame):
    '''
    Generate features from a raw taxi dataframe.
    '''
    df = df[df.fare_amount > 0]  # avoid divide-by-zero
    df['tip_fraction'] = df.tip_amount / df.fare_amount
    
    df['pickup_weekday'] = df.tpep_pickup_datetime.dt.weekday
    df['pickup_weekofyear'] = df.tpep_pickup_datetime.dt.isocalendar().week
    df['pickup_hour'] = df.tpep_pickup_datetime.dt.hour
    df['pickup_week_hour'] = (df.pickup_weekday * 24) + df.pickup_hour
    df['pickup_minute'] = df.tpep_pickup_datetime.dt.minute
    df = df[features + [y_col]].astype(float).fillna(-1)
    
    return df
    
taxi_train = prep_df(taxi)

In [ ]:
taxi_train.head()

# Train a model

Setting `nthread=-1` tells xgboost to use all available cores on this machine to parallelize model training

In [ ]:
import xgboost

xgb_reg = xgboost.XGBRegressor(
    objective="reg:squarederror",
    tree_method='approx',
    learning_rate=0.1,
    max_depth=8,
    n_estimators=100,
    nthread=-1
)

In [ ]:
%%time
_ = xgb_reg.fit(taxi_train[features], y=taxi_train[y_col])

## Save model

In [ ]:
import cloudpickle

with open(f'{MODEL_PATH}/xgboost.pkl', 'wb') as f:
    cloudpickle.dump(xgb_reg, f)

## Calculate metrics on test set

Use a different month for test set

In [ ]:
taxi_test = pd.read_csv(
    s3.open('s3://nyc-tlc/trip data/yellow_tripdata_2019-02.csv', mode='rb'),
    parse_dates=['tpep_pickup_datetime', 'tpep_dropoff_datetime']
).sample(frac=0.01, replace=False)

taxi_test = prep_df(taxi_test)

In [ ]:
from sklearn.metrics import mean_squared_error

preds = xgb_reg.predict(taxi_test[features])
mean_squared_error(taxi_test[y_col], preds, squared=False)